In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from torchvision.transforms import functional as F
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
from skimage.metrics import peak_signal_noise_ratio as psnr

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
class DenoisingDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.image_paths = [os.path.join(root_dir, img) for img in os.listdir(root_dir) 
                            if img.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))]
        self.transform = transform
    
    def add_gaussian_noise(self, image):
        noise_level = np.random.uniform(0.02, 0.1)
        noisy_image = image + noise_level * torch.randn_like(image)
        return torch.clip(noisy_image, 0., 1.)

    def add_salt_pepper_noise(self, image, prob=0.02):
        noisy_image = image.clone()
        num_pixels = image.numel()
        num_pepper = int(prob * num_pixels * 0.5)
        pepper_coords = (torch.randint(0, image.shape[1], (num_pepper,)), torch.randint(0, image.shape[2], (num_pepper,)))
        noisy_image[:, pepper_coords[0], pepper_coords[1]] = 0
        num_salt = int(prob * num_pixels * 0.5)
        salt_coords = (torch.randint(0, image.shape[1], (num_salt,)), torch.randint(0, image.shape[2], (num_salt,)))
        noisy_image[:, salt_coords[0], salt_coords[1]] = 1
        return noisy_image

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        noisy_image = self.add_gaussian_noise(image)
        noisy_image = self.add_salt_pepper_noise(noisy_image)
        return noisy_image, image

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

train_dataset = DenoisingDataset(root_dir='archive/images/train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)


In [ ]:
class ContextGuidedBlock(nn.Module):
    def __init__(self, nIn, nOut, dilation_rate=2):
        super(ContextGuidedBlock, self).__init__()
        self.conv1x1 = nn.Conv2d(nIn, nOut // 2, kernel_size=1)
        self.conv3x3 = nn.Conv2d(nOut // 2, nOut, kernel_size=3, padding=dilation_rate, dilation=dilation_rate)
        self.bn = nn.BatchNorm2d(nOut)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.conv1x1(x)
        x = self.conv3x3(x)
        x = self.bn(x)
        return self.relu(x)

class ContextModule(nn.Module):
    def __init__(self, nIn, nOut):
        super(ContextModule, self).__init__()
        self.conv_dil1 = nn.Conv2d(nIn, nOut, kernel_size=3, padding=1, dilation=1)
        self.conv_dil2 = nn.Conv2d(nIn, nOut, kernel_size=3, padding=2, dilation=2)
        self.conv_dil3 = nn.Conv2d(nIn, nOut, kernel_size=3, padding=3, dilation=3)
        self.bn = nn.BatchNorm2d(nOut)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x1 = self.conv_dil1(x)
        x2 = self.conv_dil2(x)
        x3 = self.conv_dil3(x)
        x = x1 + x2 + x3
        x = self.bn(x)
        return self.relu(x)

class CGNet(nn.Module):
    def __init__(self):
        super(CGNet, self).__init__()
        self.block1 = ContextGuidedBlock(3, 32)
        self.block2 = ContextGuidedBlock(32, 64)
        self.res_block1 = self._make_res_block(64, 64)
        self.context_module = ContextModule(64, 64)
        self.res_block2 = self._make_res_block(64, 64)
        self.conv_final = nn.Conv2d(64, 3, kernel_size=1)
    
    def _make_res_block(self, nIn, nOut):
        return nn.Sequential(
            nn.Conv2d(nIn, nOut, kernel_size=3, padding=1),
            nn.BatchNorm2d(nOut),
            nn.ReLU(),
            nn.Conv2d(nOut, nOut, kernel_size=3, padding=1),
            nn.BatchNorm2d(nOut),
            nn.ReLU()
        )

    def forward(self, x):
        x1 = self.block1(x)
        x2 = self.block2(x1)
        x2 = self.res_block1(x2)
        x_context = self.context_module(x2)
        x3 = self.res_block2(x_context)
        output = self.conv_final(x3)
        return output

model = CGNet().to(device)


In [ ]:
import os
import glob

for file in glob.glob("archive/images/train/Thumbs.db"):
    os.remove(file)
print("Tous les fichiers Thumbs.db ont été supprimés.")

class NIMAModel(nn.Module):
    def __init__(self):
        super(NIMAModel, self).__init__()
        self.base_model = models.mobilenet_v2(weights='IMAGENET1K_V2')
        self.base_model.classifier = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(self.base_model.last_channel, 10)
        )
    
    def forward(self, x):
        return self.base_model(x)

nima_model = NIMAModel().to(device)
nima_model.eval()

def calculate_nima_score(image_tensor):
    """
    Calcule le score NIMA après un pré-traitement pour améliorer la qualité perçue.
    """
    enhanced_image = enhance_image(image_tensor)
    with torch.no_grad():
        scores = nima_model(enhanced_image.unsqueeze(0).to(device))
        scores = torch.softmax(scores, dim=1)
        nima_score = torch.sum(scores * torch.arange(1, 11).to(device)).item()
    return nima_score
def enhance_image(image):
    """
    Pré-traitement pour améliorer la qualité esthétique avant d'utiliser NIMA.
    """
    image = F.adjust_saturation(image, 1.8)
    image = F.adjust_contrast(image, 1.5)
    image = F.adjust_brightness(image, 1.2)
    
    sharpen_kernel = torch.tensor(
        [[[[0, -1, 0], [-1, 5, -1], [0, -1, 0]]]], 
        dtype=torch.float32, 
        device=device
    )
    channels = []
    for c in range(3):
        channel = image[c:c+1].unsqueeze(0)
        sharpened_channel = torch.nn.functional.conv2d(channel, sharpen_kernel, padding=1)
        channels.append(sharpened_channel)
    enhanced_image = torch.cat(channels, dim=1)
    return torch.clip(enhanced_image, 0., 1.)



In [ ]:
# 3. Entraînement du modèle CGNet avec supervision multi-niveaux
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 15

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for noisy_imgs, clean_imgs in tqdm(train_loader):
        noisy_imgs = noisy_imgs.to(device)
        clean_imgs = clean_imgs.to(device)
        
        optimizer.zero_grad()
        
        outputs, intermediate = model(noisy_imgs)
        
        loss1 = criterion(outputs, clean_imgs)  
        loss2 = criterion(intermediate, clean_imgs)  
        loss = loss1 + 0.5 * loss2  
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

torch.save(model.state_dict(), 'cgnet_complete.pth')
print("Modèle sauvegardé.")


In [ ]:
class NIMAModel(nn.Module):
    def __init__(self):
        super(NIMAModel, self).__init__()
        self.base_model = models.mobilenet_v2(weights='IMAGENET1K_V2')
        self.base_model.classifier = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(self.base_model.last_channel, 10)
        )
    
    def forward(self, x):
        return self.base_model(x)

nima_model = NIMAModel().to(device)
nima_model.eval()

def calculate_nima_score(image_tensor):
    with torch.no_grad():
        scores = nima_model(image_tensor.unsqueeze(0).to(device))
        scores = torch.softmax(scores, dim=1)
        return torch.sum(scores * torch.arange(1, 11).to(device)).item()


In [ ]:
def show_sample_with_metrics(model, dataset, index):
    """
    Afficher les images bruitées, originales, et débruitées avec les scores PSNR et NIMA.
    """
    model.eval()
    noisy_img, clean_img = dataset[index]

    noisy_img = noisy_img.unsqueeze(0).to(device)
    clean_img = clean_img.unsqueeze(0).to(device)
    
    with torch.no_grad():
        output, _ = model(noisy_img)
        output = output.cpu().squeeze()
    
    psnr_noisy = psnr(clean_img.squeeze().permute(1, 2, 0).cpu().numpy(), noisy_img.squeeze().permute(1, 2, 0).cpu().numpy())
    psnr_denoised = psnr(clean_img.squeeze().permute(1, 2, 0).cpu().numpy(), output.permute(1, 2, 0).numpy())
    
    nima_noisy = calculate_nima_score(noisy_img.squeeze())
    nima_clean = calculate_nima_score(clean_img.squeeze())
    nima_denoised = calculate_nima_score(output)
    
    print(f"PSNR Noisy: {psnr_noisy:.2f}, PSNR Denoised: {psnr_denoised:.2f}")
    print(f"NIMA Noisy: {nima_noisy:.2f}, NIMA Clean: {nima_clean:.2f}, NIMA Denoised: {nima_denoised:.2f}")
    
    # Préparer les images pour affichage
    noisy_img = noisy_img.squeeze().permute(1, 2, 0).cpu().numpy()
    clean_img = clean_img.squeeze().permute(1, 2, 0).cpu().numpy()
    output = output.permute(1, 2, 0).cpu().numpy()
    
    # S'assurer que les valeurs des images sont entre 0 et 1
    noisy_img = np.clip(noisy_img, 0, 1)
    clean_img = np.clip(clean_img, 0, 1)
    output = np.clip(output, 0, 1)
    
    # Afficher les images
    plt.figure(figsize=(18, 6))
    
    plt.subplot(1, 3, 1)
    plt.title(f"Noisy Image\nNIMA: {nima_noisy:.2f}")
    plt.imshow(noisy_img)
    plt.axis('off')
    
    plt.subplot(1, 3, 2)
    plt.title(f"Clean Image\nNIMA: {nima_clean:.2f}")
    plt.imshow(clean_img)
    plt.axis('off')
    
    plt.subplot(1, 3, 3)
    plt.title(f"Denoised Image\nPSNR: {psnr_denoised:.2f}, NIMA: {nima_denoised:.2f}")
    plt.imshow(output)
    plt.axis('off')
    
    plt.show()

show_sample_with_metrics(model, train_dataset, 0)
show_sample_with_metrics(model, train_dataset, 0)
show_sample_with_metrics(model, train_dataset, 1)
show_sample_with_metrics(model, train_dataset, 2)
show_sample_with_metrics(model, train_dataset, 3)
show_sample_with_metrics(model, train_dataset, 4)
show_sample_with_metrics(model, train_dataset, 5)
show_sample_with_metrics(model, train_dataset, 6)
show_sample_with_metrics(model, train_dataset, 7)
show_sample_with_metrics(model, train_dataset, 8)
show_sample_with_metrics(model, train_dataset, 9)
show_sample_with_metrics(model, train_dataset, 10)


In [ ]:
import sys
import os

# Forcer l'installation de nbformat dans l'environnement actuel du notebook
!{sys.executable} -m pip install --upgrade nbformat

import nbformat
import zipfile

def clear_notebook_output(notebook_path, output_path=None):
    """
    Nettoyer toutes les sorties d'un notebook Jupyter pour le rendre plus léger.
    """
    with open(notebook_path, 'r', encoding='utf-8') as f:
        notebook = nbformat.read(f, as_version=4)


    for cell in notebook.cells:
        if 'outputs' in cell:
            cell['outputs'] = []
        if 'execution_count' in cell:
            cell['execution_count'] = None


    output_path = output_path or notebook_path
    with open(output_path, 'w', encoding='utf-8') as f:
        nbformat.write(notebook, f)
    print(f"Nettoyage terminé ! Notebook allégé sauvegardé sous : {output_path}")

def zip_notebook(notebook_path):
    """
    Compresser le notebook nettoyé pour réduire sa taille.
    """
    zip_path = notebook_path.replace(".ipynb", ".zip")
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        zipf.write(notebook_path)
    print(f"Notebook compressé et sauvegardé sous : {zip_path}")

def delete_large_files():
    """
    Supprimer tous les fichiers volumineux tels que .pth, .h5 et les checkpoints.
    """
    for root, dirs, files in os.walk("."):
        for file in files:
            if file.endswith(".pth") or file.endswith(".h5"):
                file_path = os.path.join(root, file)
                os.remove(file_path)
                print(f"Fichier supprimé : {file_path}")


    if os.path.exists('.ipynb_checkpoints'):
        os.system("rm -rf .ipynb_checkpoints")
        print("Checkpoints supprimés.")

    print("Nettoyage des fichiers volumineux terminé !")

notebook_name = "CGNet_BSD500.ipynb"


clear_notebook_output(notebook_name, "notebook_allégé.ipynb")
zip_notebook("notebook_allégé.ipynb")
delete_large_files()

print("\n📦 Opérations terminées ! Le notebook a été nettoyé, compressé et les fichiers inutiles supprimés.")


In [ ]:
# Exemple d'utilisation
show_sample_with_metrics(model, train_dataset, 0)
show_sample_with_metrics(model, train_dataset, 1)
show_sample_with_metrics(model, train_dataset, 2)
show_sample_with_metrics(model, train_dataset, 3)
show_sample_with_metrics(model, train_dataset, 4)
show_sample_with_metrics(model, train_dataset, 5)
show_sample_with_metrics(model, train_dataset, 6)
show_sample_with_metrics(model, train_dataset, 7)
show_sample_with_metrics(model, train_dataset, 8)
show_sample_with_metrics(model, train_dataset, 9)
show_sample_with_metrics(model, train_dataset, 10)